In [1]:
import os
import pandas as pd
import uuid
import json
import vertexai
from vertexai import agent_engines
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Callable, Dict, Optional, Union, Any
# Importy z własnych modułów
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET, ANTHROPIC_API_KEY,basic_config_agent
from agents.state import AgentWorkflowState
from agents.autogen_agents import TriggerAgent,PlannerAgent,CriticAgent
from prompts import LangchainAgentsPrompts,AutoGenAgentsPrompts
from prompts_beta import PromptFactory
from agents.langgraph_nodes import * 
from agents.autogen_agent_utils import run_autogen_planning_phase
from memory.memory_bank_client import MemoryBankClient
from tools.utils import *

In [2]:
AGENT_ENGINE_NAME = "" # Zostanie wypełniona po pobraniu lub utworzeniu silnika

# Inicjalizacja głównego klienta Vertex AI
client = vertexai.Client(project=PROJECT_ID, location=LOCATION)

In [3]:
def get_or_create_agent_engine(display_name: str) :
    """
    Pobiera istniejący Agent Engine po nazwie wyświetlanej lub tworzy nowy, jeśli nie istnieje.
    """
    # 1. Pobierz listę wszystkich istniejących silników w projekcie
    all_engines = agent_engines.list()
    
    # 2. Sprawdź, czy któryś z nich ma pasującą nazwę
    for engine in all_engines:
        if engine.display_name == display_name:
            print(f"INFO: Znaleziono i połączono z istniejącym Agent Engine: '{display_name}'")
            return engine
            
    # 3. Jeśli pętla się zakończyła i nic nie znaleziono, stwórz nowy silnik
    print(f"INFO: Nie znaleziono Agent Engine o nazwie '{display_name}'. Tworzenie nowego...")
    try:
        new_engine = agent_engines.create(
            display_name=display_name
        )
        print(f"INFO: Pomyślnie utworzono nowy Agent Engine.")
        return new_engine
    except Exception as e:
        print(f"KRYTYCZNY BŁĄD: Nie można utworzyć Agent Engine. Sprawdź konfigurację i uprawnienia. Błąd: {e}")
        exit()


In [4]:
agent_engine =get_or_create_agent_engine(MEMORY_ENGINE_DISPLAY_NAME)
AGENT_ENGINE_NAME = agent_engine.resource_name
print(AGENT_ENGINE_NAME)


INFO: Znaleziono i połączono z istniejącym Agent Engine: 'memory-gamma-way'
projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744


In [5]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}

#---WYWOŁANIE AGENTÓW
trigger_agent = TriggerAgent(llm_config=main_agent_configuration, prompt=PromptFactory.for_trigger())
planner_agent = PlannerAgent(llm_config=main_agent_configuration, prompt=PromptFactory.for_planner()) # Tutaj nie przekazujemy inspiracji
critic_agent = CriticAgent(llm_config=critic_agent_configuration, prompt=PromptFactory.for_critic())

In [6]:
if __name__ == "__main__":
    
    def master_router(state: AgentWorkflowState) -> str:
        """
        Centralny router, który zarządza całym przepływem grafu.
        Zastępuje wszystkie poprzednie, rozproszone funkcje routingu.
        """
        print(f"DEBUG ROUTER: Stan wejściowy: error='{state.get('error_message') is not None}', tool='{state.get('tool_choice')}', failing='{state.get('failing_node')}'")

        # Scenariusz 1: Wystąpił błąd
        if state.get("error_message"):
            if state.get("correction_attempts", 0) >= MAX_CORRECTION_ATTEMPTS:
                print("ROUTER: Przekroczono limit prób. Eskalacja do człowieka.")
                return "human_escalation"
            print("ROUTER: Wykryto błąd. Przechodzenie do debuggera.")
            return "universal_debugger"

        # Scenariusz 2: Debugger wybrał narzędzie
        if tool_choice := state.get("tool_choice"):
            print(f"ROUTER: Debugger wybrał narzędzie '{tool_choice}'.")
            if tool_choice == "propose_code_fix":
                return "apply_code_fix"
            if tool_choice == "request_package_installation":
                return "human_approval"
            return "human_escalation" # Domyślna akcja dla nieznanego narzędzia

        # Scenariusz 3: Trwa proces naprawczy (failing_node jest ustawiony, ale error_message jest już czysty)
        if failing_node := state.get("failing_node"):
            print(f"ROUTER: Zakończono próbę naprawy. Powrót do węzła '{failing_node}'.")
            # Kluczowe: resetujemy failing_node, aby po udanej próbie graf mógł iść dalej
            state["failing_node"] = None
            return failing_node

        # Jeśli żaden z powyższych warunków nie jest spełniony, oznacza to, że graf ma kontynuować "szczęśliwą ścieżkę".
        # W tym wypadku zwracamy specjalny sygnał, a decyzję podejmie sama krawędź.
        print("ROUTER: Brak błędów i akcji naprawczych. Kontynuacja głównej ścieżki.")
        return "continue"
    
    
    files_to_exclude = {'Agents_beta (10).py','pack_project.ipynb', 'caly_projekt.txt'}
    system_source_code = read_project_source_code(".", exclude_files=files_to_exclude)

    # --- Inicjalizacja Pamięci i Uruchomienia ---
    memory_client = MemoryBankClient(client=client, agent_engine=agent_engine)
    run_id = str(uuid.uuid4())
    
    print("\n--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---")
    inspiration_prompt = ""
    dataset_signature = ""
    try:
        df_preview = pd.read_csv(INPUT_FILE_PATH, nrows=0)
        dataset_signature = memory_client.create_dataset_signature(df_preview)
        past_memories = memory_client.query_memory(
            query_text="Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych",
            scope={"dataset_signature": dataset_signature},
            top_k=3
        )
        if past_memories:
            inspirations = []
            for mem in past_memories:
                # ZMIANA: Używamy nowych, poprawnych typów wspomnień
                if mem.memory_type == MemoryType.SUCCESSFUL_WORKFLOW and 'key_planning_insight' in mem.content:
                    inspirations.append(f"SPRAWDZONY WNIOSEK Z PROCESU: {mem.content['key_planning_insight']}")
                elif mem.memory_type == MemoryType.SUCCESSFUL_FIX and 'key_takeaway' in mem.content:
                    inspirations.append(f"NAUCZKA Z NAPRAWIONEGO BŁĘDU: {mem.content['key_takeaway']}")
            if inspirations:
                inspiration_prompt = "--- INSPIRACJE Z POPRZEDNICH URUCHOMIEŃ ---\n" + "\n".join(inspirations)
                print("INFO: Pomyślnie pobrano inspiracje z pamięci.")
        else:
            print("INFO: Nie znaleziono inspiracji w pamięci dla tego typu danych.")
    except Exception as e:
        print(f"OSTRZEŻENIE: Nie udało się pobrać inspiracji z pamięci: {e}")

        
        
    active_policies = get_active_policies_from_memory(memory_client, dataset_signature)    
    
    # --- Krok 1: Faza planowania (AutoGen) ---
    final_plan, autogen_log = run_autogen_planning_phase(
        input_path=INPUT_FILE_PATH, 
        inspiration_prompt=inspiration_prompt,
        trigger_agent=trigger_agent,
        planner_agent=planner_agent,
        critic_agent=critic_agent,
        manager_agent_config=main_agent_configuration,
        active_policies=active_policies
    )
    save_autogen_conversation_log(log_content=autogen_log, file_path="reports/autogen_planning_conversation.log")

    # --- Krok 2: Faza wykonania (LangGraph) ---
    if final_plan:
        print("\n" + "="*80)
        print("### ### FAZA 2: URUCHAMIANIE WYKONANIA PLANU (LangGraph) ### ###")
        print("="*80 + "\n")
        
        workflow = StateGraph(AgentWorkflowState)
        
        # <<< ZMIANA TUTAJ: Zaktualizowana lista węzłów >>>
        nodes = [
            "schema_reader", "code_generator", "architectural_validator", 
            "data_code_executor", "universal_debugger", "apply_code_fix", 
            "human_approval", "package_installer", "human_escalation", 
            "sync_report_code","meta_auditor",
            # Nowe, wyspecjalizowane węzły raportujące:
            "summary_analyst", "plot_generator", "report_composer","pre_audit_summarizer","memory_consolidation"
        ]
        for name in nodes:
            if f"{name}_node" in globals():
                workflow.add_node(name, globals()[f"{name}_node"])
            else:
                # Poprawka: Używamy poprawnej nazwy funkcji, jeśli nazwa węzła jej nie zawiera
                workflow.add_node(name, globals()[name])

        # Ustawienie punktu wejściowego
        workflow.set_entry_point("schema_reader")

        # Definicja prostych, liniowych krawędzi
        workflow.add_edge("schema_reader", "code_generator")
        workflow.add_edge("code_generator", "architectural_validator")

        # Krawędzie warunkowe, które ZAWSZE przechodzą przez nasz nowy, centralny router
        conditional_routing_map = {
            "continue": "data_code_executor", # Domyślna następna krawędź dla tego węzła
            "universal_debugger": "universal_debugger",
            "human_escalation": "human_escalation"
        }
        workflow.add_conditional_edges("architectural_validator", master_router, conditional_routing_map)

        # Kolejne kroki głównej ścieżki - każdy z nich używa tego samego, prostego schematu
        workflow.add_conditional_edges("data_code_executor", master_router, {**conditional_routing_map, "continue": "summary_analyst"})
        workflow.add_conditional_edges("summary_analyst", master_router, {**conditional_routing_map, "continue": "plot_generator"})
        workflow.add_conditional_edges("plot_generator", master_router, {**conditional_routing_map, "continue": "report_composer"})
        workflow.add_conditional_edges("report_composer", master_router, {**conditional_routing_map, "continue": "pre_audit_summarizer"})

        # --- ŚCIEŻKI NAPRAWCZE ---
        # Po tych krokach, również wracamy do routera, aby podjął decyzję
        workflow.add_conditional_edges("apply_code_fix", master_router, conditional_routing_map)
        workflow.add_conditional_edges("package_installer", master_router, conditional_routing_map)
        workflow.add_conditional_edges("universal_debugger", master_router, {
            "apply_code_fix": "apply_code_fix",
            "human_approval": "human_approval",
            "human_escalation": "human_escalation",
            
            "universal_debugger": "human_escalation"
        })
        workflow.add_conditional_edges("human_approval", lambda s: s.get("user_approval_status"), {
            "APPROVED": "package_installer",
            "REJECTED": "universal_debugger"
        })

        # --- KOŃCOWA, JEDNOLITA ŚCIEŻKA AUDYTU ---
        workflow.add_edge("human_escalation", "pre_audit_summarizer")
        workflow.add_edge("pre_audit_summarizer", "meta_auditor")
        workflow.add_edge("meta_auditor", "memory_consolidation")
        workflow.add_edge("memory_consolidation", END)

        # ======================================================================
        # ### KROK 2: KOMPILACJA GRAFU ###
        # ======================================================================
        app = workflow.compile()
        
        app_config = {"MAIN_AGENT": MAIN_AGENT, "CODE_MODEL": CODE_MODEL, "CRITIC_MODEL": CRITIC_MODEL}
        
        initial_state = {
            "config": app_config,
            "plan": final_plan, 
            "input_path": INPUT_FILE_PATH,
            "output_path": "reports/processed_data.csv",
            "report_output_path": "reports/transformation_report.html",
            "correction_attempts": 0, 
            "correction_history": [],
            "source_code": system_source_code,
            "autogen_log": autogen_log,
            "memory_client": memory_client,
            "run_id": run_id,
            "dataset_signature": dataset_signature,
            "pending_fix_session": None,
            "active_policies": active_policies
        }
        
        langgraph_log = ""
        final_run_state = initial_state.copy()
        
        for event in app.stream(initial_state, {"recursion_limit": 50}):
            for node_name, state_update in event.items():
                if "__end__" not in node_name:
                    print(f"--- Krok: '{node_name}' ---")
                    if state_update:
                        printable_update = state_update.copy()
                        for key in ["generated_code", "corrected_code", "generated_report_code", "error_context_code", "plot_generation_code", "summary_html"]:
                            if key in printable_update and printable_update[key]:
                                print(f"--- {key.upper()} ---")
                                print(printable_update[key])
                                print("-" * (len(key) + 8))
                                del printable_update[key]
                        if printable_update:
                            print(json.dumps(printable_update, indent=2, default=str))
                        
                        log_line = f"--- Krok: '{node_name}' ---\n{json.dumps(state_update, indent=2, default=str)}\n"
                        langgraph_log += log_line
                        final_run_state.update(state_update)
                    else:
                        print("  [INFO] Węzeł zakończył pracę bez aktualizacji stanu.")
                    print("-" * 20 + "\n")

        save_langgraph_execution_log(log_content=langgraph_log, file_path="reports/langgraph_execution.log")

        final_run_state['langgraph_log'] = langgraph_log
        meta_auditor_node(final_run_state)

        print("\n\n--- ZAKOŃCZONO PRACĘ GRAFU I AUDYT ---")
    else:
        print("Proces zakończony. Brak planu do wykonania.")

INFO: MemoryBankClient gotowy do pracy z silnikiem: projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744

--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---
INFO: Odpytuję pamięć semantycznie z zapytaniem 'Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych' w zakresie {'dataset_signature': 'ae1568fe7dae11d4bacd0c21ed718503'}


/home/jupyter/olga_zydziak/version_beta/multiagent_system/memory/memory_bank_client.py:95: ExperimentalWarning: The Vertex SDK GenAI agent engines module is experimental, and may change in future versions.
  memories_iterator = self.client.agent_engines.retrieve_memories(


udany plan: id='444d2b32-3209-4862-b197-b6abed33fd0c' run_id='a243bb20-6961-4255-94c3-bddf69c68294' timestamp=datetime.datetime(2025, 8, 7, 11, 20, 53, 648589) memory_type=<MemoryType.SUCCESSFUL_WORKFLOW: 'SUCCESSFUL_WORKFLOW'> dataset_signature='ae1568fe7dae11d4bacd0c21ed718503' source_node='memory_consolidation_node' content={'workflow_summary': 'Pomyślnie przygotowano dane do modelowania poprzez szczegółowe czyszczenie, inżynierię cech i zastosowanie zaawansowanej, dwutorowej strategii skalowania.', 'key_planning_insight': 'Zastosowanie podwójnej strategii skalowania — `RobustScaler` dla danych skośnych i z outlierami (`Transaction_Amount`) oraz `StandardScaler` dla pozostałych — było kluczową decyzją, która zapewniła optymalne przygotowanie cech numerycznych.', 'key_execution_insight': 'Strukturyzacja kodu w dedykowanej funkcji `process_data` z solidną obsługą błędów `try-except` okazała się kluczowa do spełnienia wymogów architektonicznych i zapewnienia odporności skryptu.', 'fina